# Analysis of Covid-19 Risk factors by an opinion extraction strategy


### Intro

We are a group of engineers at Atos/Bull.

### Goals

Our goal was to **overcomes the problem of finding quickly different opinions** about a given subjet. In fact, it can be very difficult to quickly get reliable information: many different points of view are represented in the medias as well as in the scientific litterature.

Instead of simply returning the most closest sentences to the query, we chose to **extract the different opinions**, which can be shared by the different groups of people working on a subject.

Showing an exemple of our results on Chloroquine in this notebook, might motivate one to read further on the topic or to use our tool for the various tasks of the challenge.

This notebook proove according to us, that opinion extraction can be useful to the community.

### How it works 

#### Database creation

All titles, abstracts and body texts of the dataset have been [inserted into an SQLite DB](https://github.com/MrMimic/covid-19-kaggle/blob/master/src/main/python/c19/database_utilities.py#L186) (only english articles for the moment). 

They have been preprocessed by using the [method we developed](https://github.com/MrMimic/covid-19-kaggle/blob/master/src/main/python/c19/text_preprocessing.py#L21). It will lower and stem the text, remove stopwords, remove numeric values, etc.

A [word2vec](https://radimrehurek.com/gensim/models/word2vec.html) embedding and a [TF-IDF](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) models have been trained on this pre-processed corpus. Briefly, these models allow to get a fixed-length vector to represent each word of the corpus (word2vec) and to weight each word regarding it's frequency among all the corpus and in each document (TF-IDF). The result is a parquet table, [stored on Github](https://github.com/MrMimic/covid-19-kaggle/blob/master/resources/global_df_w2v_tfidf.parquet), containing for each word a float vector and a TF-IDF score. 

![Table header](https://raw.githubusercontent.com/MrMimic/covid-19-kaggle/master/images/header_w2v_tfidf.png "Logo Title Text 1")

The file can be re-generated in more or less 30 minutes on a 8 vCPU machine by using [this script](https://github.com/MrMimic/covid-19-kaggle/blob/master/src/main/scripts/train_w2v.py).

Each sentence from the corpus have been pre-processed and [vectorized](https://github.com/MrMimic/covid-19-kaggle/blob/master/src/main/python/c19/embedding.py#L65). To do so, each pre-processed word from a sentence is represented by its vector and weithed by the TF-IDF score. Then, all vectors from the different words composing the sentence are averaged ([Mean of Word Embeddings](https://books.google.fr/books?id=tBxrDwAAQBAJ&pg=PA95&lpg=PA95&dq=mean+of+word+embedding+MOWE&source=bl&ots=7laX_HWKS0&sig=ACfU3U2DvGwGI6Bs4HTkX0_oP7Nf3UTP2A&hl=en&sa=X&ved=2ahUKEwiXguOJ9tjoAhX3D2MBHS6mAzoQ6AEwCnoECA0QKA#v=onepage&q=mean%20of%20word%20embedding%20MOWE&f=false)). All these pre-processed sentences are [stored in base](https://github.com/MrMimic/covid-19-kaggle/blob/master/src/main/python/c19/text_preprocessing.py#L151).

#### Query matching

The query is first [vectorised](https://github.com/MrMimic/covid-19-kaggle/blob/master/src/main/python/c19/query_matching.py#L57) by using the same strategy and tool as explained above. The cosine similarity of this sentence [versus all stored sentences](https://github.com/MrMimic/covid-19-kaggle/blob/master/src/main/python/c19/query_matching.py#L127) vectors is then computed. Briefly, it allows to check how each sentence of the dataset is close from the query. Only the top-k sentences are returned (filtered either by minimal distance or by a fixed number of top-k sentences).

All these top-k closest sentences are then clusterised by a Kmean algorithm. These clusters will represent the different opinions found about the query.

Only the closest sentence from each centroid is returned (*ie*, the sentence reflecting the most the opinion on this subject).

### What's cool

- The trained embedding is not generic. Even if pre-trained models found on the Internet work well, the context of covid-19 and the kind of sentences to be processed make a locally trained embedding better.
- Code is highly optimisez for RAM and rapid processing. Only the resulting DB weights gigabytes.

### What's not

- The database containing all sentences weights more than 25Go. It is thus unusable on Kaggle. To overcome this issue, we had to select randomly 10 sentences from the body.

### What's next

Version 2.0 of this work should be released soon. To come:

- Ranking best papers of opinions clusters by using a citation network (see our current work on that [NOTEBOOK](https://www.kaggle.com/baptistemetge/citation-network) ) and maybe some extra metadata (the authors, the journals,etc.). 
- Auto-estimate K for the number of opinions.
- Auto-test the code on Github with unitary tests on the methods.
- Etc ;)

And during the round #2, we would to develop:

- A multi-lingual search (maybe with trained embedding on different languages instead of just translating the query).
- Use a larger pre-trained embedding (on the same corpus but maybe with some data augmentation from PubMed about given subjects).

### Usage

Queries from the different tasks have been reformulated and [stored in Github](https://github.com/MrMimic/covid-19-kaggle/blob/master/resources/queries.json). All of them have been sent to the pipeline and the result are store in markdown format here.

For this notebook, we will focus on a given task and try to answer by using our tool.

### Setup

The library can be easily [installed from github](https://github.com/MrMimic/covid-19-kaggle/blob/master/setup.py).

In [1]:
# Install custom library from Github
!pip install -q --no-warn-conflicts git+https://github.com/MrMimic/covid-19-kaggle

import os
# Custom lib installed from github
from c19 import database_utilities, text_preprocessing, embedding, query_matching, parameters

# Ugly dependencies warnings
import warnings
warnings.filterwarnings("ignore")

Then, the parameters are loaded ([full explaination of the parameters](https://github.com/MrMimic/covid-19-kaggle/blob/master/src/main/python/c19/parameters.py)).

In [2]:
params = parameters.Parameters(
    first_launch=True,
    database=parameters.Database(
        local_path="local_database.sqlite",
        kaggle_data_path=os.path.join(os.sep, "kaggle", "input", "CORD-19-research-challenge")
    ),
    preprocessing=parameters.PreProcessing(
        max_body_sentences=10,
        stem_words=False
    ),
    query=parameters.Query(
        top_k_sentences_distance=0.8,
        filtering_method="distance"
    )
)

We construct the DB.

In [3]:
database_utilities.create_db_and_load_articles(
    db_path=params.database.local_path,
    kaggle_data_path=params.database.kaggle_data_path,
    first_launch=params.first_launch,
    load_body=params.preprocessing.load_text_body)

41361 articles to be prepared.


PRE-PROCESSING: 100%|██████████| 41361/41361 [01:20<00:00, 514.30it/s] 


Took 1.34 min to prepare 41361 articles for insertion.
Took 0.18 min to insert 41361 articles (SQLite DB: local_database.sqlite).


The pre-trained embeddings are loaded from GIthub

In [4]:
embedding_model = embedding.Embedding(
    parquet_embedding_path=params.embedding.local_path,
    embeddings_dimension=params.embedding.dimension,
    sentence_embedding_method=params.embedding.word_aggregation_method,
    weight_vectors=params.embedding.weight_with_tfidf)

Took 0.57 min to load 48539 Word2Vec vectors (embedding dim: 100).


The sentences are pre-processed

In [5]:
text_preprocessing.pre_process_and_vectorize_texts(
    embedding_model=embedding_model,
    db_path=params.database.local_path,
    first_launch=params.first_launch,
    stem_words=params.preprocessing.stem_words,
    remove_num=params.preprocessing.remove_numeric,
    batch_size=params.preprocessing.batch_size,
    max_body_sentences=params.preprocessing.max_body_sentences)

41361 files to pre-process (42 batches of 1000 articles).


PRE-PROCESSING: 100%|██████████| 42/42 [34:00<00:00, 48.58s/it]  


Took 34.5 min to pre-process 42 batches of articles.
Took 0.38 min to insert 613148 sentences (SQLite DB: local_database.sqlite).


Ready to be used.

### Analyse: covid-19 risk factors study

In [6]:
full_sentences_db = query_matching.get_sentences_data(
        db_path=params.database.local_path)

Queries will be match versus 613140 sentences (1.69 minutes to load).


In [7]:
query = "What do we know about Chloroquine to treat covid-19 induced by coronavirus?"

In [8]:
closest_sentences_df = query_matching.get_k_closest_sentences(
    query=query,
    all_sentences=full_sentences_db,
    embedding_model=embedding_model,
    number_threshold=params.query.top_k_sentences_number,
    distance_threshold=params.query.top_k_sentences_distance,
    filtering_method=params.query.filtering_method)

Took 0.22 minutes to process the query (443 sentences kept by distance filtering).


In [9]:
closest_sentences_df = query_matching.clusterise_sentences(
    k_closest_sentences_df=closest_sentences_df,
    number_of_clusters=3)

Took 0.24 seconds to clusterise 443 closest sentences.


In [10]:
closest_sentences_df.sort_values(by="is_closest", ascending=False).head(10)

,paper_doi,section,raw_sentence,sentence,vector,distance,cluster,is_closest
320,10.1111/jcmm.13467,abstract,Although antiviral drugs are available for the...,"[""although"", ""antiviral"", ""drugs"", ""available""...","[-0.6874068137481052, -0.7672865114260059, 0.5...",0.809898,0,True
4,10.1016/j.dsx.2020.03.011,body,In the absence of any known efficient therapy ...,"[""absence"", ""known"", ""efficient"", ""therapy"", ""...","[-1.2439228987417703, -2.0248827386088615, 0.8...",0.887833,1,True
35,10.1016/s1473-3099(03)00806-5,body,The use of chloroquine in combination with oth...,"[""use"", ""chloroquine"", ""combination"", ""antiret...","[-0.7287883106587858, -1.484976677645821, 0.25...",0.859595,2,True
291,10.1097/cm9.0000000000000797,title,Repurposing of clinically approved drugs for t...,"[""repurposing"", ""clinically"", ""approved"", ""dru...","[-0.5995144221422676, -0.7631291321702953, 0.3...",0.811909,0,False
302,10.1099/jgv.0.000309,abstract,In mouse models of EBOV disease there are conf...,"[""mouse"", ""models"", ""ebov"", ""disease"", ""confli...","[-0.33989102021762957, -1.4382496967795593, -0...",0.810822,2,False
301,10.1016/s0140-6736(10)60357-1,abstract,These data show the potential of RNA interfere...,"[""data"", ""show"", ""potential"", ""rna"", ""interfer...","[-0.07484730531915117, -1.2519306647688682, -0...",0.810981,0,False
300,10.1038/nm0503-479,body,We also do not know whether SARS is caused by ...,"[""also"", ""know"", ""whether"", ""sars"", ""caused"", ...","[-0.020479399977647708, -1.1668740411243568, -...",0.811139,2,False
299,10.3389/fmicb.2019.03079,abstract,"Currently, there are no vaccines or therapeuti...","[""currently"", ""vaccines"", ""therapeutic"", ""drug...","[-0.2752155307130031, -0.6325520597285712, 0.3...",0.811207,0,False
298,10.1099/jgv.0.000309,abstract,These results advocate that chloroquine should...,"[""results"", ""advocate"", ""chloroquine"", ""consid...","[-0.77465973534322, -1.8164638308488823, 0.260...",0.811225,1,False
297,10.1128/cmr.00045-07,abstract,Summary: Though several antivirals have been d...,"[""summary"", ""though"", ""several"", ""antivirals"",...","[-0.5380644737652062, -0.9421896836808804, 0.3...",0.811253,0,False


In [11]:
# There is 3 Clusters:
closest_sentences_df["cluster"].value_counts()

0    211
2    146
1     86
Name: cluster, dtype: int64

In [12]:
for index, row in closest_sentences_df[closest_sentences_df["is_closest"] == True].iterrows():
    print(f"Cluster : {row.cluster}")
    print(f"{row.raw_sentence}")
    print()

Cluster : 1
In the absence of any known efficient therapy and because of the situation of a "public-health emergency", many drugs have been tried recently in the treatment for COVID-19 that includes a low-cost antimalarial drug chloroquine and its derivative hydroxychloroquine (HCQ), along with several other antiviral drugs.

Cluster : 2
The use of chloroquine in combination with other antiretrovirals is theoretically supported by the observation that chloroquine also shows anti-HIV activity in vitro towards isolates from patients in therapeutic failure with a multidrug-resistant profile.

Cluster : 0
Although antiviral drugs are available for the treatment of influenza infection, it is an urgent requirement to develop new antiviral drugs regarding the emergence of drug‐resistant viruses.

